In [2]:
import pyranges as pr
import pandas as pd
import numpy as np
from helpers import get_terminal_regions, _df_update_3p, _df_update_5p, add_region_number
import os

In [9]:
# Get Zeng gene names

outdir = "processed/isoform_specific/regions/"
path_zeng_bed = os.path.join(outdir, "zeng_pas_fig2c.updated.bed.tsv")
zeng_bed = pr.PyRanges(pd.read_csv(path_zeng_bed, sep="\t"))
zeng_gn = set(zeng_bed.gene_name_common)
zeng_bed

,Chromosome,Start,End,Name,Score,Strand,gene_name_common,pas_id
0,chr1,629997,629998,MTND2P28,.,+,MTND2P28,chr1:629997:629998:+:MTND2P28
1,chr1,630367,630368,MTND2P28,.,+,MTND2P28,chr1:630367:630368:+:MTND2P28
2,chr1,854387,854388,LINC01128,.,+,LINC01128,chr1:854387:854388:+:LINC01128
3,chr1,859444,859445,LINC01128,.,+,LINC01128,chr1:859444:859445:+:LINC01128
4,chr1,1011462,1011463,ISG15,.,+,ISG15,chr1:1011462:1011463:+:ISG15
...,...,...,...,...,...,...,...,...
16096,chrY,18932449,18932450,TTTY14,.,-,TTTY14,chrY:18932449:18932450:-:TTTY14
16097,chrY,19691944,19691945,NaN,.,-,ENSG00000260197,chrY:19691944:19691945:-:ENSG00000260197
16098,chrY,19692490,19692491,NaN,.,-,ENSG00000260197,chrY:19692490:19692491:-:ENSG00000260197
16099,chrY,19703866,19703867,KDM5D,.,-,KDM5D,chrY:19703866:19703867:-:KDM5D


In [7]:
# load in BED file of PAPA cryptics (to get gene names and ids)
papa_cryp_bed = pr.read_bed("data/2023-12-15_all.last_exons.cryptic.bed")

# get le_ids for cryptics (+ their annotated partners)
# e.g. ENSG00000213468.7_1|FIRRE|bleedthrough|cryptic	
papa_name_spl = papa_cryp_bed.Name.str.split("\\|", expand=True)
cryp_le_ids = set(papa_name_spl[0])
cryp_gn = set(papa_name_spl[1])
print(f"Number of PAPA cryptics - {len(cryp_le_ids)}")
print(cryp_le_ids)
papa_cryp_bed

Number of PAPA cryptics - 284
{'ENSG00000164609.10_5', 'ENSG00000153094.24_3', 'ENSG00000134909.19_1', 'ENSG00000261221.4_1', 'ENSG00000146733.14_1', 'ENSG00000118922.18_1', 'ENSG00000119509.13_2', 'ENSG00000244405.8_3', 'ENSG00000185352.9_1', 'ENSG00000247708.8_2', 'ENSG00000136144.12_3', 'ENSG00000214562.15_2', 'ENSG00000247199.6_1', 'ENSG00000149577.16_3', 'ENSG00000126767.18_1', 'ENSG00000135090.14_5', 'ENSG00000184347.15_7', 'ENSG00000111450.14_1', 'ENSG00000151692.15_6', 'ENSG00000181090.21_5', 'ENSG00000155897.10_1', 'ENSG00000155093.19_2', 'ENSG00000132485.14_3', 'ENSG00000107960.11_1', 'ENSG00000128274.17_2', 'ENSG00000105926.16_1', 'ENSG00000114904.13_3', 'ENSG00000004866.22_2', 'ENSG00000154319.16_2', 'ENSG00000153094.24_2', 'ENSG00000281566.3_1', 'ENSG00000198797.7_1', 'ENSG00000254602.2_1', 'ENSG00000140750.17_4', 'ENSG00000148814.18_3', 'ENSG00000105948.13_3', 'ENSG00000065526.12_4', 'ENSG00000242686.4_2', 'ENSG00000125676.20_2', 'ENSG00000032219.19_2', 'ENSG00000132394.1

,Chromosome,Start,End,Name,Score,Strand
0,chr1,15939295,15940456,ENSG00000065526.12_3|SPEN|proximal|cryptic,.,+
1,chr1,15939295,15940477,ENSG00000065526.12_4|SPEN|distal|cryptic,.,+
2,chr1,177280411,177282422,ENSG00000198797.7_1|BRINP2|proximal|cryptic,.,+
3,chr1,177280411,177284690,ENSG00000198797.7_2|BRINP2|distal|cryptic,.,+
4,chr1,76871267,76871821,ENSG00000117069.15_2|ST6GALNAC5|spliced|cryptic,.,+
...,...,...,...,...,...,...
279,chrX,123600573,123601338,ENSG00000125676.20_3|THOC2|distal|cryptic,.,-
280,chrX,17835910,17837395,ENSG00000131831.18_1|RAI2|spliced|cryptic,.,-
281,chrX,108267641,108269152,ENSG00000197565.17_1|COL4A6|spliced|cryptic,.,-
282,chrX,40653641,40654363,ENSG00000180182.11_3|MED14|bleedthrough|cryptic,.,-


In [4]:
# Combined reference of novel + annotated last exons used for quantification
papa_gtf = pr.read_gtf("data/novel_ref_combined.quant.last_exons.gtf")
papa_gtf

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,...,region_rank,Start_ref,End_ref,transcript_id_ref,3p_extension_length,event_type,ref_gene_id,ref_gene_name,le_number,le_id
0,chr1,.,exon,24419640,24420128,.,+,.,ENSG00000001461.17,NIPAL3,...,last,24419290,24419640,ENST00000003912.7,488.0,internal_exon_extension,ENSG00000001461.17,NIPAL3,1.0,ENSG00000001461.17_1
1,chr1,.,exon,24421813,24422110,.,+,.,ENSG00000001461.17,NIPAL3,...,last,"24419640,24419640,24419640","24440171,24440171,24433103","ENST00000358028.8,ENST00000374399.9,ENST000000...","NULL,NULL,NULL",internal_exon_spliced,ENSG00000001461.17,NIPAL3,2.0,ENSG00000001461.17_2
2,chr1,.,exon,24454053,24454824,.,+,.,ENSG00000001461.17,NIPAL3,...,last,"24453504,24453504","24456137,24456137","ENST00000003912.7,ENST00000374399.9","NULL,NULL",internal_exon_spliced,ENSG00000001461.17,NIPAL3,3.0,ENSG00000001461.17_3
3,chr1,.,exon,24464120,24466378,.,+,.,ENSG00000001461.17,NIPAL3,...,last,24464025,24464120,ENST00000003912.7,2258.0,internal_exon_extension,ENSG00000001461.17,NIPAL3,4.0,ENSG00000001461.17_4
4,chr1,.,exon,24468985,24472976,.,+,.,ENSG00000001461.17,NIPAL3,...,last,"24464120,24464120","24468985,24468985","ENST00000003912.7,ENST00000374399.9","NULL,NULL",last_exon_spliced,ENSG00000001461.17,NIPAL3,5.0,ENSG00000001461.17_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129925,chrY,.,exon,21880075,21880652,.,-,.,ENSG00000244395.6,RBMY1D,...,last,"21880652,21880652,21880652","21881063,21881063,21881063","ENST00000382653.6,ENST00000382680.5,ENST000004...","NULL,NULL,NULL",last_exon_spliced,ENSG00000244395.6,RBMY1D,1.0,ENSG00000244395.6_1
129926,chrY,.,exon,21880075,21880652,.,-,.,ENSG00000244395.6,RBMY1D,...,last,"21880652,21880652,21880652","21881063,21881063,21881063","ENST00000382653.6,ENST00000382680.5,ENST000004...","NULL,NULL,NULL",last_exon_spliced,ENSG00000244395.6,RBMY1D,1.0,ENSG00000244395.6_1
129927,chrY,.,exon,21880307,21880652,.,-,.,ENSG00000244395.6,RBMY1D,...,last,"21880652,21880652,21880652","21881063,21881063,21881063","ENST00000382653.6,ENST00000382680.5,ENST000004...","NULL,NULL,NULL",last_exon_spliced,ENSG00000244395.6,RBMY1D,1.0,ENSG00000244395.6_1
129928,chrY,.,exon,21038288,21039044,.,-,.,ENSG00000254488.1,ENSG00000254488,...,last,21039044,21042268,ENST00000527562.1,NULL,last_exon_spliced,ENSG00000254488.1,ENSG00000254488,1.0,ENSG00000254488.1_1


In [21]:
# some gene names are duplicated in the ref_gene_name column

def remove_duplicates_preserve_order(s):
    seen = set()
    result = []
    for item in s.split(','):
        if item not in seen:
            seen.add(item)
            result.append(item)
    return ','.join(result)

# Apply the function to the series
papa_gtf.ref_gene_name = papa_gtf.ref_gene_name.apply(remove_duplicates_preserve_order)
papa_gtf



,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,...,region_rank,Start_ref,End_ref,transcript_id_ref,3p_extension_length,event_type,ref_gene_id,ref_gene_name,le_number,le_id
0,chr1,.,exon,24419640,24420128,.,+,.,ENSG00000001461.17,NIPAL3,...,last,24419290,24419640,ENST00000003912.7,488.0,internal_exon_extension,ENSG00000001461.17,NIPAL3,1.0,ENSG00000001461.17_1
1,chr1,.,exon,24421813,24422110,.,+,.,ENSG00000001461.17,NIPAL3,...,last,"24419640,24419640,24419640","24440171,24440171,24433103","ENST00000358028.8,ENST00000374399.9,ENST000000...","NULL,NULL,NULL",internal_exon_spliced,ENSG00000001461.17,NIPAL3,2.0,ENSG00000001461.17_2
2,chr1,.,exon,24454053,24454824,.,+,.,ENSG00000001461.17,NIPAL3,...,last,"24453504,24453504","24456137,24456137","ENST00000003912.7,ENST00000374399.9","NULL,NULL",internal_exon_spliced,ENSG00000001461.17,NIPAL3,3.0,ENSG00000001461.17_3
3,chr1,.,exon,24464120,24466378,.,+,.,ENSG00000001461.17,NIPAL3,...,last,24464025,24464120,ENST00000003912.7,2258.0,internal_exon_extension,ENSG00000001461.17,NIPAL3,4.0,ENSG00000001461.17_4
4,chr1,.,exon,24468985,24472976,.,+,.,ENSG00000001461.17,NIPAL3,...,last,"24464120,24464120","24468985,24468985","ENST00000003912.7,ENST00000374399.9","NULL,NULL",last_exon_spliced,ENSG00000001461.17,NIPAL3,5.0,ENSG00000001461.17_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129925,chrY,.,exon,21880075,21880652,.,-,.,ENSG00000244395.6,RBMY1D,...,last,"21880652,21880652,21880652","21881063,21881063,21881063","ENST00000382653.6,ENST00000382680.5,ENST000004...","NULL,NULL,NULL",last_exon_spliced,ENSG00000244395.6,RBMY1D,1.0,ENSG00000244395.6_1
129926,chrY,.,exon,21880075,21880652,.,-,.,ENSG00000244395.6,RBMY1D,...,last,"21880652,21880652,21880652","21881063,21881063,21881063","ENST00000382653.6,ENST00000382680.5,ENST000004...","NULL,NULL,NULL",last_exon_spliced,ENSG00000244395.6,RBMY1D,1.0,ENSG00000244395.6_1
129927,chrY,.,exon,21880307,21880652,.,-,.,ENSG00000244395.6,RBMY1D,...,last,"21880652,21880652,21880652","21881063,21881063,21881063","ENST00000382653.6,ENST00000382680.5,ENST000004...","NULL,NULL,NULL",last_exon_spliced,ENSG00000244395.6,RBMY1D,1.0,ENSG00000244395.6_1
129928,chrY,.,exon,21038288,21039044,.,-,.,ENSG00000254488.1,ENSG00000254488,...,last,21039044,21042268,ENST00000527562.1,NULL,last_exon_spliced,ENSG00000254488.1,ENSG00000254488,1.0,ENSG00000254488.1_1


In [28]:
# exclude cryptic genes or those input from quant-seq
papa_gtf_filt = papa_gtf.subset(lambda df: ~df.ref_gene_name.isin(cryp_gn.union(zeng_gn)))[["ref_gene_name", "le_id", "event_type", "ref_gene_id", "gene_id", "transcript_id", "le_number"]]
papa_gtf_filt

,Chromosome,Start,End,Strand,ref_gene_name,le_id,event_type,ref_gene_id,gene_id,transcript_id,le_number
0,chr1,11820361,11820779,+,CLCN6,ENSG00000011021.23_1,internal_exon_spliced,ENSG00000011021.23,ENSG00000011021.23,ENST00000376497.7,1.0
1,chr1,11840371,11841016,+,CLCN6,ENSG00000011021.23_2,first_exon_extension,ENSG00000011021.23,ENSG00000011021.23,ENST00000312413.10,2.0
2,chr1,11841207,11843143,+,CLCN6,ENSG00000011021.23_2,first_exon_extension,ENSG00000011021.23,ENSG00000011021.23,ENST00000312413.10,2.0
3,chr1,11840371,11841016,+,CLCN6,ENSG00000011021.23_2,first_exon_extension,ENSG00000011021.23,ENSG00000011021.23,ENST00000346436.11,2.0
4,chr1,11841207,11843130,+,CLCN6,ENSG00000011021.23_2,first_exon_extension,ENSG00000011021.23,ENSG00000011021.23,ENST00000346436.11,2.0
...,...,...,...,...,...,...,...,...,...,...,...
65420,chrY,21880075,21880652,-,RBMY1D,ENSG00000244395.6_1,last_exon_spliced,ENSG00000244395.6,ENSG00000244395.6,ENST00000382653.6,1.0
65421,chrY,21880075,21880652,-,RBMY1D,ENSG00000244395.6_1,last_exon_spliced,ENSG00000244395.6,ENSG00000244395.6,ENST00000382680.5,1.0
65422,chrY,21880307,21880652,-,RBMY1D,ENSG00000244395.6_1,last_exon_spliced,ENSG00000244395.6,ENSG00000244395.6,ENST00000418956.2,1.0
65423,chrY,21038288,21039044,-,ENSG00000254488,ENSG00000254488.1_1,last_exon_spliced,ENSG00000254488.1,ENSG00000254488.1,ENST00000527562.1,1.0


In [36]:
# create merged intervals per last exon ID
merged_le_id = papa_gtf_filt.merge(strand=True, by="le_id")
merged_le_id

,Chromosome,Start,End,Strand,le_id
0,chr1,196673163,196673856,+,ENSG00000000971.17_1
1,chr1,196673962,196675913,+,ENSG00000000971.17_1
2,chr1,196701317,196701566,+,ENSG00000000971.17_2
3,chr1,196747110,196747504,+,ENSG00000000971.17_3
4,chr1,171099804,171100735,+,ENSG00000007933.13_1
...,...,...,...,...,...
32367,chrY,21903617,21904194,-,ENSG00000242389.9_1
32368,chrY,24183433,24183505,-,ENSG00000244231.1_1
32369,chrY,21880075,21880652,-,ENSG00000244395.6_1
32370,chrY,21038288,21039044,-,ENSG00000254488.1_1


In [37]:
# return minimal metadata to each i
metadata_le_id = papa_gtf_filt.as_df()[["le_id", "ref_gene_name", "event_type", "le_number"]].astype({"le_number": float}).astype({"le_number": 'int32'})
merged_le_id = merged_le_id.apply(lambda df: df.merge(metadata_le_id, on="le_id", how="left"))
merged_le_id

,Chromosome,Start,End,Strand,le_id,ref_gene_name,event_type,le_number
0,chr1,196673163,196673856,+,ENSG00000000971.17_1,CFH,internal_exon_extension,1
1,chr1,196673163,196673856,+,ENSG00000000971.17_1,CFH,internal_exon_extension,1
2,chr1,196673962,196675913,+,ENSG00000000971.17_1,CFH,internal_exon_extension,1
3,chr1,196673962,196675913,+,ENSG00000000971.17_1,CFH,internal_exon_extension,1
4,chr1,196701317,196701566,+,ENSG00000000971.17_2,CFH,internal_exon_spliced,2
...,...,...,...,...,...,...,...,...
74369,chrY,21880075,21880652,-,ENSG00000244395.6_1,RBMY1D,last_exon_spliced,1
74370,chrY,21880075,21880652,-,ENSG00000244395.6_1,RBMY1D,last_exon_spliced,1
74371,chrY,21880075,21880652,-,ENSG00000244395.6_1,RBMY1D,last_exon_spliced,1
74372,chrY,21038288,21039044,-,ENSG00000254488.1_1,ENSG00000254488,last_exon_spliced,1


## 3'Exts - generate distal bins

In [71]:
# get genes with distal 3'utr extensions
# extract last exons, then check event types
merged_le_id_le = get_terminal_regions(merged_le_id.assign("Feature", lambda df: pd.Series(["exon"]*len(df), index=df.index)),
                                       id_col="ref_gene_name", region_number_col="le_number")

novel_3ext_gn = set(merged_le_id_le.subset(lambda df: df.event_type == "last_exon_extension").ref_gene_name)
len(novel_3ext_gn)

1563

In [72]:
# subset to genes with 3'exts
merged_le_id_3ext = merged_le_id.subset(lambda df: df.ref_gene_name.isin(novel_3ext_gn))

# extract last_exons
merged_le_id_3ext_le = get_terminal_regions(merged_le_id_3ext.assign("Feature", lambda df: pd.Series(["exon"]*len(df), index=df.index)), id_col="ref_gene_name", region_number_col="le_number")

# extract non-last exons
merged_le_id_3ext_nles = merged_le_id_3ext.subset(lambda df: ~df.le_id.isin(set(merged_le_id_3ext_les.le_id)))

merged_le_id_3ext_le

,Chromosome,Start,End,Strand,le_id,ref_gene_name,event_type,le_number,Feature
0,chr1,63436822,63440437,+,ENSG00000088035.18_3,ALG6,last_exon_extension,3,exon
1,chr1,150995259,150995885,+,ENSG00000143412.10_2,ANXA9,last_exon_extension,2,exon
2,chr1,1495484,1498495,+,ENSG00000160072.20_2,ATAD3B,last_exon_extension,2,exon
3,chr1,190480424,190481783,+,ENSG00000237457.6_2,BRINP3-DT,last_exon_extension,2,exon
4,chr1,178547163,178551764,+,ENSG00000213057.5_2,C1orf220,last_exon_extension,2,exon
...,...,...,...,...,...,...,...,...,...
1559,chrX,155478419,155491666,-,ENSG00000185973.12_5,TMLHE,last_exon_extension,5,exon
1560,chrX,101006828,101010707,-,ENSG00000188917.15_2,TRMT2B,last_exon_extension,2,exon
1561,chrX,47444691,47449474,-,ENSG00000147124.13_2,ZNF41,last_exon_extension,2,exon
1562,chrY,17551679,17553954,-,ENSG00000230663.1_3,FAM224B,last_exon_extension,3,exon


In [73]:
merged_le_id_3ext_nles

,Chromosome,Start,End,Strand,le_id,ref_gene_name,event_type,le_number
0,chr1,11820361,11820779,+,ENSG00000011021.23_1,CLCN6,internal_exon_spliced,1
1,chr1,11840371,11841016,+,ENSG00000011021.23_2,CLCN6,first_exon_extension,2
2,chr1,11840371,11841016,+,ENSG00000011021.23_2,CLCN6,first_exon_extension,2
3,chr1,11840371,11841016,+,ENSG00000011021.23_2,CLCN6,first_exon_extension,2
4,chr1,11840371,11841016,+,ENSG00000011021.23_2,CLCN6,first_exon_extension,2
...,...,...,...,...,...,...,...,...
9095,chrX,3817527,3817778,-,ENSG00000285756.2_6,ENSG00000285756,last_exon_spliced,6
9096,chrY,304528,305197,-,ENSG00000178605.13_PAR_Y_1,GTPBP6,last_exon_spliced,1
9097,chrY,304528,305197,-,ENSG00000178605.13_PAR_Y_1,GTPBP6,last_exon_spliced,1
9098,chrY,17574730,17575367,-,ENSG00000230663.1_1,FAM224B,internal_exon_spliced,1


In [74]:
# Extract distal bins
# define distal specific region (ONLY for UTRs)
merged_le_id_3ext_les_uniq = merged_le_id_3ext_le.subtract(merged_le_id_3ext_nles, strandedness="same")
merged_le_id_3ext_les_uniq

,Chromosome,Start,End,Strand,le_id,ref_gene_name,event_type,le_number,Feature
0,chr1,63437133,63440437,+,ENSG00000088035.18_3,ALG6,last_exon_extension,3,exon
1,chr1,150995634,150995885,+,ENSG00000143412.10_2,ANXA9,last_exon_extension,2,exon
2,chr1,1497848,1498495,+,ENSG00000160072.20_2,ATAD3B,last_exon_extension,2,exon
3,chr1,190480735,190481783,+,ENSG00000237457.6_2,BRINP3-DT,last_exon_extension,2,exon
4,chr1,178548889,178551764,+,ENSG00000213057.5_2,C1orf220,last_exon_extension,2,exon
...,...,...,...,...,...,...,...,...,...
1485,chrX,155478419,155489010,-,ENSG00000185973.12_5,TMLHE,last_exon_extension,5,exon
1486,chrX,101006828,101009345,-,ENSG00000188917.15_2,TRMT2B,last_exon_extension,2,exon
1487,chrX,47444691,47445878,-,ENSG00000147124.13_2,ZNF41,last_exon_extension,2,exon
1488,chrY,17551679,17552799,-,ENSG00000230663.1_3,FAM224B,last_exon_extension,3,exon


## Parse regions into featureCounts compliant GTF

- each interval reported with 'exon', 'gene' and 'transcript' assigned to Feature column (i.e. reported 3*)
- gene_id, transcript_id and le_id minimal attributes ()
  - assign le_id to all, except for 3'exts where add a suffix


### 3'exts

In [75]:
# distal specific bins = le_id + suffix
distal_suffix = "__distal"
merged_le_id_3ext_les_uniq.gene_id = merged_le_id_3ext_les_uniq.le_id + distal_suffix
merged_le_id_3ext_les_uniq.transcript_id = merged_le_id_3ext_les_uniq.gene_id
merged_le_id_3ext_les_uniq.type = merged_le_id_3ext_les_uniq.Feature
merged_le_id_3ext_les_uniq = merged_le_id_3ext_les_uniq.drop("event_type").apply(lambda df: df.rename(columns={"ref_gene_name": "gene_name"}))

# add entries with transcript and gene in the feature column
grs = [merged_le_id_3ext_les_uniq.assign("Feature", lambda df: pd.Series([feat]*len(df), index=df.index)) for feat in ["transcript", "gene"]]
# combine into single gr
merged_le_id_3ext_les_uniqs = pr.concat([merged_le_id_3ext_les_uniq, *grs]).sort()
merged_le_id_3ext_les_uniqs

,Chromosome,Start,End,Strand,le_id,gene_name,le_number,Feature,gene_id,transcript_id,type
0,chr1,1497848,1498495,+,ENSG00000160072.20_2,ATAD3B,2,gene,ENSG00000160072.20_2__distal,ENSG00000160072.20_2__distal,exon
1,chr1,1497848,1498495,+,ENSG00000160072.20_2,ATAD3B,2,exon,ENSG00000160072.20_2__distal,ENSG00000160072.20_2__distal,exon
2,chr1,1497848,1498495,+,ENSG00000160072.20_2,ATAD3B,2,transcript,ENSG00000160072.20_2__distal,ENSG00000160072.20_2__distal,exon
3,chr1,2505532,2505811,+,ENSG00000149527.18_2,PLCH2,2,gene,ENSG00000149527.18_2__distal,ENSG00000149527.18_2__distal,exon
4,chr1,2505532,2505811,+,ENSG00000149527.18_2,PLCH2,2,exon,ENSG00000149527.18_2__distal,ENSG00000149527.18_2__distal,exon
...,...,...,...,...,...,...,...,...,...,...,...
4465,chrY,302836,304528,-,ENSG00000178605.13_PAR_Y_2,GTPBP6,2,transcript,ENSG00000178605.13_PAR_Y_2__distal,ENSG00000178605.13_PAR_Y_2__distal,exon
4466,chrY,302836,304528,-,ENSG00000178605.13_PAR_Y_2,GTPBP6,2,gene,ENSG00000178605.13_PAR_Y_2__distal,ENSG00000178605.13_PAR_Y_2__distal,exon
4467,chrY,17551679,17552799,-,ENSG00000230663.1_3,FAM224B,3,exon,ENSG00000230663.1_3__distal,ENSG00000230663.1_3__distal,exon
4468,chrY,17551679,17552799,-,ENSG00000230663.1_3,FAM224B,3,transcript,ENSG00000230663.1_3__distal,ENSG00000230663.1_3__distal,exon


In [76]:
# full 3'UTRs
merged_le_id_3ext_le.gene_id = merged_le_id_3ext_le.le_id
merged_le_id_3ext_le.transcript_id = merged_le_id_3ext_le.le_id
merged_le_id_3ext_le = merged_le_id_3ext_le.drop("event_type").apply(lambda df: df.rename(columns={"ref_gene_name": "gene_name"}))
merged_le_id_3ext_le.type = merged_le_id_3ext_le.Feature
merged_le_id_3ext_le

# add entries with transcript and gene in the feature column
grs = [merged_le_id_3ext_le.assign("Feature", lambda df: pd.Series([feat]*len(df), index=df.index)) for feat in ["transcript", "gene"]]
# combine into single gr
merged_le_id_3ext_les = pr.concat([merged_le_id_3ext_le, *grs]).sort()
merged_le_id_3ext_les

,Chromosome,Start,End,Strand,le_id,gene_name,le_number,Feature,gene_id,transcript_id,type
0,chr1,1495484,1498495,+,ENSG00000160072.20_2,ATAD3B,2,exon,ENSG00000160072.20_2,ENSG00000160072.20_2,exon
1,chr1,1495484,1498495,+,ENSG00000160072.20_2,ATAD3B,2,transcript,ENSG00000160072.20_2,ENSG00000160072.20_2,exon
2,chr1,1495484,1498495,+,ENSG00000160072.20_2,ATAD3B,2,gene,ENSG00000160072.20_2,ENSG00000160072.20_2,exon
3,chr1,2503921,2505811,+,ENSG00000149527.18_2,PLCH2,2,transcript,ENSG00000149527.18_2,ENSG00000149527.18_2,exon
4,chr1,2503921,2505811,+,ENSG00000149527.18_2,PLCH2,2,exon,ENSG00000149527.18_2,ENSG00000149527.18_2,exon
...,...,...,...,...,...,...,...,...,...,...,...
4687,chrY,302836,305197,-,ENSG00000178605.13_PAR_Y_2,GTPBP6,2,transcript,ENSG00000178605.13_PAR_Y_2,ENSG00000178605.13_PAR_Y_2,exon
4688,chrY,302836,305197,-,ENSG00000178605.13_PAR_Y_2,GTPBP6,2,gene,ENSG00000178605.13_PAR_Y_2,ENSG00000178605.13_PAR_Y_2,exon
4689,chrY,17551679,17553954,-,ENSG00000230663.1_3,FAM224B,3,exon,ENSG00000230663.1_3,ENSG00000230663.1_3,exon
4690,chrY,17551679,17553954,-,ENSG00000230663.1_3,FAM224B,3,transcript,ENSG00000230663.1_3,ENSG00000230663.1_3,exon


### All other events

In [77]:
merged_le_id_other = merged_le_id.subset(lambda df: ~df.ref_gene_name.isin(novel_3ext_gn))
merged_le_id_other

# full 3'UTRs
merged_le_id_other.gene_id = merged_le_id_other.le_id
merged_le_id_other.transcript_id = merged_le_id_other.le_id
merged_le_id_other = merged_le_id_other.drop("event_type").apply(lambda df: df.rename(columns={"ref_gene_name": "gene_name"}))
merged_le_id_other.Feature = "exon"
merged_le_id_other.type = merged_le_id_other.Feature

# add entries with transcript and gene in the feature column
grs = [merged_le_id_other.assign("Feature", lambda df: pd.Series([feat]*len(df), index=df.index)) for feat in ["transcript", "gene"]]
# combine into single gr
merged_le_id_others = pr.concat([merged_le_id_other, *grs]).sort()
merged_le_id_others

,Chromosome,Start,End,Strand,le_id,gene_name,le_number,gene_id,transcript_id,Feature,type
0,chr1,365170,366052,+,ENSG00000236601.2_1,ENSG00000236601,1,ENSG00000236601.2_1,ENSG00000236601.2_1,exon,exon
1,chr1,365170,366052,+,ENSG00000236601.2_1,ENSG00000236601,1,ENSG00000236601.2_1,ENSG00000236601.2_1,exon,exon
2,chr1,365170,366052,+,ENSG00000236601.2_1,ENSG00000236601,1,ENSG00000236601.2_1,ENSG00000236601.2_1,gene,exon
3,chr1,365170,366052,+,ENSG00000236601.2_1,ENSG00000236601,1,ENSG00000236601.2_1,ENSG00000236601.2_1,gene,exon
4,chr1,365170,366052,+,ENSG00000236601.2_1,ENSG00000236601,1,ENSG00000236601.2_1,ENSG00000236601.2_1,transcript,exon
...,...,...,...,...,...,...,...,...,...,...,...
177712,chrY,25378299,25378878,-,ENSG00000228786.5_1,LINC00266-4P,1,ENSG00000228786.5_1,ENSG00000228786.5_1,gene,exon
177713,chrY,25378299,25378878,-,ENSG00000228786.5_1,LINC00266-4P,1,ENSG00000228786.5_1,ENSG00000228786.5_1,exon,exon
177714,chrY,57201142,57202020,-,ENSG00000185203.12_PAR_Y_1,WASIR1,1,ENSG00000185203.12_PAR_Y_1,ENSG00000185203.12_PAR_Y_1,gene,exon
177715,chrY,57201142,57202020,-,ENSG00000185203.12_PAR_Y_1,WASIR1,1,ENSG00000185203.12_PAR_Y_1,ENSG00000185203.12_PAR_Y_1,transcript,exon


## Combine all and output to GTF

In [78]:
comb_gtf_out = pr.concat([merged_le_id_others, merged_le_id_3ext_les, merged_le_id_3ext_les_uniqs]).sort().drop_duplicate_positions()
comb_gtf_out

,Chromosome,Start,End,Strand,le_id,gene_name,le_number,gene_id,transcript_id,Feature,type
0,chr1,365170,366052,+,ENSG00000236601.2_1,ENSG00000236601,1,ENSG00000236601.2_1,ENSG00000236601.2_1,exon,exon
1,chr1,594234,594574,+,ENSG00000235146.2_1,ENSG00000235146,1,ENSG00000235146.2_1,ENSG00000235146.2_1,exon,exon
2,chr1,804966,805270,+,ENSG00000237491.10_1,LINC01409,1,ENSG00000237491.10_1,ENSG00000237491.10_1,transcript,exon
3,chr1,807323,808378,+,ENSG00000237491.10_2,LINC01409,2,ENSG00000237491.10_2,ENSG00000237491.10_2,gene,exon
4,chr1,809621,810063,+,ENSG00000237491.10_3,LINC01409,3,ENSG00000237491.10_3,ENSG00000237491.10_3,gene,exon
...,...,...,...,...,...,...,...,...,...,...,...
30638,chrY,25030900,25031222,-,ENSG00000185894.8_1,BPY2C,1,ENSG00000185894.8_1,ENSG00000185894.8_1,gene,exon
30639,chrY,25063082,25064083,-,ENSG00000228296.1_1,TTTY4C,1,ENSG00000228296.1_1,ENSG00000228296.1_1,gene,exon
30640,chrY,25182276,25182587,-,ENSG00000223641.2_1,TTTY17C,1,ENSG00000223641.2_1,ENSG00000223641.2_1,gene,exon
30641,chrY,25378299,25378878,-,ENSG00000228786.5_1,LINC00266-4P,1,ENSG00000228786.5_1,ENSG00000228786.5_1,exon,exon


In [79]:
out_gtf_path = os.path.join(outdir, "papa_other_feature_counts.gtf")
comb_gtf_out.to_gtf(out_gtf_path)